# Profiler Viewer In Notebook

In [ ]:
import numpy as np
import datetime

In [ ]:
from whylogs import get_or_create_session
from whylogs.core import DatasetProfile
from whylogs.viz import display_profile

## Data for Logging

In [ ]:
!pip install faker

In [ ]:
from collections import OrderedDict
from faker import Faker

locales = OrderedDict([
    ('en-US', 1),
    ('fr-FR', 2),
    ('ja_JP', 2),
])
fake= Faker(locales)
distribution = np.concatenate((np.random.normal(0.1, 0.1, 500), np.random.normal(0.6, 0.2, 500)))

### 📝 Log it


In [ ]:
session = get_or_create_session()
def profile_generator():
    with session.logger("mytestytest",dataset_timestamp=datetime.datetime(2021, 6, 2)) as logger:
        for _ in range(500):
            logger.log({"uniform_integers": np.random.randint(0,50)})
            logger.log({"strings": fake.name()})
            logger.log({"mixture_distribution": np.random.choice(distribution, 1)[0]})
            logger.log({"nulls": None})
        logger.log({"moah_data":1})
        logger.log({"moah_data":1})
        logger.log({"moah_data":5})
        return logger.profile
profile=profile_generator()

## 👀 open viewer

By calling `display_profile`, it will display a summary of your profiled dataset.

In [ ]:
display_profile(profiles=[profile])


### Logging reference profile

In [ ]:
reference_profile=profile_generator() 

### Add reference profile to viewer and open

In [ ]:
display_profile(profiles=[profile], reference_profiles=[reference_profile])